# Histogram of EM Data with Automatic Min/Max Visualization 

This notebook is for exploration of automatic computation of Min/Max parameters for adjust with image intestity range used to visualized electron microscopy image data.

In [ ]:
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from pathlib import Path
%matplotlib notebook

In [ ]:
fname = "data/ave_2013-1220-dA30_5-BSC-1_7.mrc"
img = sitk.ReadImage(fname)
a_img = sitk.GetArrayViewFromImage(img).flatten()
bins = 2**np.iinfo(a_img.dtype).bits
h, b = np.histogram(a_img, bins=bins, range=(np.iinfo(a_img.dtype).min, np.iinfo(a_img.dtype).max + 1))

In [ ]:
def histogram_robust_stats(hist, bin_edges):

    assert (len(hist) + 1 == len(bin_edges))
    results = {}

    mids = 0.5 * (bin_edges[1:] + bin_edges[:-1])
    cs = np.cumsum(hist)
    median_idx = np.searchsorted(cs, cs[-1] / 2.0)
    results["median"] = mids[median_idx]
    
    mad_mids, mad_h = zip(*sorted(zip(np.abs(mids - results["median"] ), hist)))
    mad_cs = np.cumsum(mad_h)
    mad_idx = np.searchsorted(mad_cs, mad_cs[-1] / 2.0)

    
    results["mad"] = mad_mids[mad_idx]
    
    return results

In [ ]:
print(histogram_robust_stats(h,b))

median = np.median(a_img)
mad = np.median(np.abs(a_img-median))
print(f"median: {median}\nMAD: {mad}")

In [ ]:
def rescale_min_max(img, vmin, vmax):
    ss_filter = sitk.ShiftScaleImageFilter()
    ss_filter.SetShift( -float(vmin) )
    ss_filter.SetScale( 255.0/(vmax-vmin) )
    ss_filter.SetOutputPixelType(sitk.sitkUInt8)
    return ss_filter.Execute(img)

In [ ]:

def plot_range_options(img, sigma_mult=3, mad_mult=4, percentile_crop=10, bins=1024, fig_filename=None, title=None):


    fa_img = sitk.GetArrayViewFromImage(img).flatten()
    median = np.median(fa_img)
    mad = np.median(np.abs(fa_img-median))
    mean = np.mean(fa_img)
    sigma = np.sqrt(np.var(fa_img))


    mad_range =  (median-mad_mult*mad, median+mad_mult*mad)
    percentile_range = (np.percentile(fa_img,percentile_crop*.5), np.percentile(fa_img,100-percentile_crop*.5))

    
    sigma_range = (mean-sigma_mult*sigma, 
                   mean+sigma_mult*sigma)
    
    min_max_range = (np.min(fa_img), np.max(fa_img))

    fig = plt.figure(figsize=(8, 8), dpi=240)
    if title:
        fig.suptitle(title, fontsize=8)
    ax = fig.add_subplot(2, 4, (1, 4))

    x = np.linspace(min(fa_img), max(fa_img), bins)
    plt.plot(x, norm.pdf(x, mean, sigma), color='k', linestyle='--', alpha=0.5, label=f'Fit Gaussian')

    plt.hist( fa_img, bins=bins, density=True, color='b')

    plt.axvline(x=mad_range[0], color='r', alpha=0.5, label=f'{mad_mult} Median Absolute Deviation')
    plt.axvline(x=mad_range[1], color='r', alpha=0.5)
    plt.axvline(x=median, color='r', alpha=0.5, linestyle='--', label=f'Median')
    print(f"Median: {median} {mad_mult} Median Absolute Deviation: {mad_range}")


    plt.axvline(x=percentile_range[0], color='g', alpha=0.5, label=f'Middle {100-percentile_crop} Percentile')
    plt.axvline(x=percentile_range[1], color='g', alpha=0.5)
    print(f"Middle {100-percentile_crop} Percentile: {percentile_range}" )

    plt.axvline(x=sigma_range[0], color='y', alpha=0.5, label=f'{sigma_mult} Sigma')
    plt.axvline(x=sigma_range[1], color='y', alpha=0.5)
    print(f"{sigma_mult} Sigma: {sigma_range}" )
    
    plt.axvline(x=min_max_range[0], color='k', alpha=0.5, label=f'Min/Max')
    plt.axvline(x=min_max_range[1], color='k', alpha=0.5)
    print(f"Min/Max: {min_max_range}" )


    
   

    plt.legend(fontsize=5)
    plt.title("Data Ranges and Histogram")

    if img.GetSize()[0] >= img.GetSize()[2]:
        if img.GetSize()[1] >= img.GetSize()[2]:
            img_slice = img[:,:, img.GetSize()[2]//2]
        else:
            img_slice = img[:,img.GetSize()[1]//2,:]
    else:
        if img.GetSize()[1] >= img.GetSize()[0]:
            img_slice = img[img.GetSize()[0]//2,:,:]
        else:
            img_slice = img[:,img.GetSize()[1]//2,:]
           
    
    ax = fig.add_subplot(2, 4, 5)
    plt.imshow(sitk.GetArrayViewFromImage(img_slice), vmin=mad_range[0], vmax=mad_range[1], cmap='gray')
    ax.axes.xaxis.set_visible(False)
    ax.axes.yaxis.set_visible(False)
    plt.title(f'{mad_mult} Median Absolute Deviation', color='r', fontsize=5)
    
    ax = fig.add_subplot(2, 4, 6)
    plt.imshow(sitk.GetArrayViewFromImage(img_slice), vmin=percentile_range[0], vmax=percentile_range[1], cmap='gray')
    ax.axes.xaxis.set_visible(False)
    ax.axes.yaxis.set_visible(False)
    plt.title(f'Middle {100-percentile_crop} Percentile', color='g', fontsize=5)
    
    ax = fig.add_subplot(2, 4, 7)
    plt.imshow(sitk.GetArrayViewFromImage(img_slice), vmin=sigma_range[0], vmax=sigma_range[1], cmap='gray')
    ax.axes.xaxis.set_visible(False)
    ax.axes.yaxis.set_visible(False)
    plt.title(f'{sigma_mult} Sigma', color='y', fontsize=5)
    
    
    ax = fig.add_subplot(2, 4, 8)
    #plt.imshow(sitk.GetArrayViewFromImage(rescale_min_max(img_slice, *min_max_range)), cmap='gray')
    plt.imshow(sitk.GetArrayViewFromImage(img_slice), vmin=min_max_range[0], vmax=min_max_range[1], cmap='gray')
    ax.axes.xaxis.set_visible(False)
    ax.axes.yaxis.set_visible(False)
    plt.title(f'Data Min/Max', color='k', fontsize=5)
    
    plt.show()
    if fig_filename:
        plt.savefig(fig_filename)

In [ ]:
for path in Path.cwd().glob("data/*.mrc"):
    img = sitk.ReadImage(str(path))
    print(path)
    plot_range_options(img, mad_mult=5, percentile_crop=4, bins=1024,  fig_filename=f"{path.with_suffix('')}_fig.pdf", title=str(path.name))

In [ ]:
plot_range_options?

In [ ]:
a_img = sitk.GetArrayViewFromImage(img).flatten()
bins = 2**np.iinfo(a_img.dtype).bits

h, b = np.histogram(list(), bins=bins, range=(np.iinfo(a_img.dtype).min, np.iinfo(a_img.dtype).max + 1))
       

In [ ]:
def stream_build_histogram(filename:str, histogram_bin_edges, extract_axis = 1, density = False):
    reader = sitk.ImageFileReader()
    reader.SetFileName(filename)
    reader.ReadImageInformation()
      
    extract_index = [0]*reader.GetDimension()

    extract_size = list(reader.GetSize())
    extract_size[extract_axis] = 0
    reader.SetExtractSize(extract_size)

    h = np.zeros(len(histogram_bin_edges)-1, dtype=np.int64)
    
    for i in range(reader.GetSize()[extract_axis]):
        extract_index[extract_axis] = i
        reader.SetExtractIndex(extract_index)
        img = reader.Execute()
        
        # accumulate histogram density/weights
        h += np.histogram( sitk.GetArrayViewFromImage(img).flatten(), bins=histogram_bin_edges, density=density)[0]
    
    return h, np.array(np.histogram_bin_edges)

In [ ]:
bin_edges=range(np.iinfo(a_img.dtype).min, np.iinfo(a_img.dtype).max + 2)
h, b = stream_build_histogram(input_filename, bin_edges)